# TF的MNIST入门

本文适合对机器学习和TF都不太了解的新手。

如果读者已经了解MNIST和Softmax回归的相关知识，可以直接跳往[TF的MNIST提高](./TF的MNIST提高.ipynb)。

很多人生平第一次编程，就是打印“hello world”，MNIST好比机器学习的Hello-World。

MNIST是一种入门级的计算机视觉数据集，包含各种手写数字图片。

![](./图表/MNIST.png "手写数字图片示例")

每一张图包含一个数字以及对应的标签，该标签保存真实的数值。上面四张图的标签分别是“5、0、4、1”。

要讲述的就是训练一个机器学习模型来预测图片里的数字。

目的不是设计一个世界一流的复杂模型，尽管之后给出的代码可以实现一流的预测模型，而是要介绍如何使用TF来训练模型。

从一个很简单的数学模型起步，叫做Softmax回归。

对应的实现代码很短，且真正有意思的内容只有三行。但是，理解这些代码的设计思想非常重要：TF工作流程和机器学习的基本概念。本文会很详细地介绍这些代码的实现原理。

## MNIST数据集

来自[Yann LeCun](http://yann.lecun.com/exdb/mnist/)，这里提供代码用于自动下载和配置这个数据集。

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


下载的数据被分为两部分：

- 六万行的训练数据集（mnist.train）
- 一万行的测试数据集（mnist.test）

这样切分很重要，在机器学习模型设计时必须有一个单独的测试数据集、不用于训练而用于评估整个模型的性能，从而更容易把设计的模型推广到别的数据集上（泛化）。

正如前面条的一样，每个MNIST数据单元由两部分组成：

- 包含一个手写数字的图片
- 图片对应的标签标识真值

将这些图片设为 $xs$ ，把这些标签设为 $ys$ 。

训练数据集和测试数据集都包含 $xs$ 和 $ys$ ，比如训练数据集的图片是mnist.train.images，训练数据集的标签是mnist.train.labels。

**每张图片包含28*28个像素点**。

可以用像素矩阵来表示一张图片。

![](./图表/MNIST-Matrix.png "数字图片的矩阵表示")

把这个矩阵展开成一个向量，长度是784（28*28）。

如何展开这个矩阵（行列甚至元素的顺利）不重要，只要保证各个图片采用相同的方式展开。

站在这个角度考虑，MNIST数据集的图片就是784维向量空间中的点集，并且拥有比较[复杂的结构](http://colah.github.io/posts/2014-10-Visualizing-MNIST/ "目测MNIST：降维测试")（此类数据的可视化是计算密集型的）。

展开矩阵会丢失维度结构信息，这显然不理想，最优秀的计算机视觉方法会挖掘并利用这些结构信息，后续会介绍，本文暂时忽略，只介绍简单数学模型，Softmax回归不会利用这些结构信息。

因此，在MNIST训练数据集中，mnist.train.images是一个形状为 $[60000, 784]$ 的张量，第一个维度用来索引图片，第二个维度用来索引某张图片中的像素点。在此张量中，每个元素都表示某张图片里某个像素的强度值，介于 $[0, 1]$ 之间。

![](./图表/mnist-train-xs.png)

相应的MNIST数据集的标签式介于 $[0, 9]$ 的数字，用来描述给定图片里表示的数字。

为了用于本文，标签数据是“one-hot”向量，即某一位数字是 $1$ 其余位都是 $0$ 。

所以在本文中，数字 $n$ 将表示成一个只在第n维（从0起）为 $1$ 的十维向量。

例如，标签 $0$ 表示为 $[1,0,0,0,0,0,0,0,0,0]$ 。

因此，mnist.train.labels是一个 $[60000, 10]$ 的矩阵。

![](./图表/mnist-train-ys.png)

接下来，着手构建模型！

## Softmax回归

都知道MNIST的每一张图片都表示一个 $[0, 9]$ 的自然数字。

希望得到给定图片每个数字的概率，如模型推测真值是9的图片包含9的概率是80%、包含8的概率是5%（因为8和9上半部分的圈近似），包含别的数字的概率更小。

这是应用Softmax回归的经典案例。

**Softmax可以用来给不同的对象分配概率，即便以后训练更加精细的模型时，最后一步也需要用Softmax来分配概率**。

- 第一步

    为了得到给定图片属于某个特定数字的证据，对图片像素值加权求和，若其中某个像素具有很强的证据表明该图片不属于这个特定数字，相应的权值为负数，想法是正数。
    
    下图展示某模型学习到的图片上每个像素对特定数字的权值，红色为负数、蓝色为正数。
    
    ![](./图表/softmax-weights.png)
    
    还需要添加一个额外的偏置，因为输入往往会带有一些无关的干扰量，因此对于给定的图片 $x$ 代表数字 $i$ 的证据表示为： $$evidence_i = \sum_jW_{i,j}x_j + b_i$$
    
    其中， $W_i$ 代表权重， $b_i$ 代表数字 $i$ 的偏置， $j$ 代表给定图片 $x$ 的像素索引（用于像素加权求和）。
    
    接着用Softmax把这些证据转换成概率 $y$ ： $$y=softmax(evidence)$$
    
    这里的 $softmax$ 可以看成一个激励函数（或链接函数），把定义的线性函数的输出转换成想要的格式，也就是关于十个数字的概率分布。这样，给定一张图，它对于每个数字的吻合度可以被Softmax转换成一个概率值。
    
    Softmax的定义为： $$softmax(x)=normalize(\exp(x))$$
    
    展开等式右侧的子式，可以得到： $$softmax(x)_i=\frac{\exp(x_i)}{\sum_j\exp(x_j)}$$
    
    更多时候把Softmax模型函数定义为前一种形式：把输入值当成幂指数求值，接着正则化这些结果值。
    
    这个幂运算表明：更大的证据对应更大的假设模型里面的乘数权重值；反之，拥有更少的证据意味着在假设模型里拥有更小的乘数系数。
    
    假设模型里的权值不可以是零或负数。
    
    Softmax然后会正则化这些权重值，使得总和等于一，以此构造有效地概率分布。
    
    *更多有关Softmax函数的解释，[参考Michael Nieslen的书](http://neuralnetworksanddeeplearning.com/)，其中有Softmax的交互式可视化阐述*。